# PROGRES - Mini-Projet 2 
# API Web 

Fabien Mathieu - fabien.mathieu@normalesup.org

Sébastien Tixeuil - Sebastien.Tixeuil@lip6.fr

## Etudiants:

- Thierry Ung

- Jack Thay

Le but de ce mini-projet est d'utiliser (entre autres) la bibliothèque Python bottle, pour 
proposer d’une part une API spécifique au site http://dblp.uni-trier.de/ qui 
regroupe l'ensemble des publications scientifiques en informatique, et d’autre part un site web 
qui permet d’utiliser l’API précédente. 

Le site dblp propose l'ensemble des publications sous la forme d'un fichier Xml. Il faut donc 
télécharger ce fichier et utiliser les données qu'il contient afin de créer votre API. Dans la suite, 
on appelle publication, un élément de type `article`, `inproceedings`, `proceedings`, 
`book`, `incollection`, `phdthesis`, ou `mastersthesis`.

Le fichier Xml se trouve à l'adresse http://dblp.uni-trier.de/xml/. Il doit être analysé 
pour récupérer chaque publication (quel que soit son type). L’API demandée porte seulement 
sur les champs `author`, `title`, `year`, `journal`, et `booktitle` (les autres champs 
peuvent donc être ignorés). On peut remarquer que pour une publication, soit le champ 
`journal`, soit le champ `booktitle` est défini (`booktitle` correspond au nom de la 
conférence dans laquelle est publié l’article, `journal` correspond à la revue scientifique dans 
laquelle est publié l’article).

Note: En raison de la quantité limitée de mémoire RAM disponible sur certains ordinateurs, il 
est possible de ne considérer que les publications des 5 dernières années (voire moins, mais 
ce paramètre doit pouvoir être modifié dans le code source).

**RAPPEL :** stackoverflow est votre ami... **Mais** si vous utilisez quelque chose que vous trouvez sur Internet, vous devez citer votre source **ET** ajouter des explications. Ne copiez pas des blocs de code entiers sans comprendre, sans expliquer, sans citer.

La taille conséquente du fichier xml empêche de lancer l’analyse du fichier car l'ordinateur n'a pas assez de ram. Il est donc nécessaire de réaliser un pré-traitement en codant un programme permettant de générer plusieurs sous-fichiers plus légers. Pour cela, notre code divise le fichier xml principal en sous-fichiers contenant un nombre spécifique de publications. A l’aide des balises contenues dans le fichier xml, nous avons pu les compter et fixer a 500 000 le nombre de publications par fichier. Nous nous sommes donc retrouvées avec plusieurs fichiers xml. On choisis de travailler sur un des ces sous-fichiers.

In [ ]:
import os
from pathlib import *

## Block of code used to cut "dblp.xml" in smaller parts, it may take a few dozen of minutes

publications = ['</article>', '</inproceedings>', '</proceedings>','</book>', '</incollection>', '</phdthesis>','</mastersthesis>']
publications1 = ['<article', '<inproceedings', '<proceedings','<book', '<incollection', '<phdthesis','<mastersthesis']
end = ['</dblp>']
entete = ['<?xml version="1.0" encoding="ISO-8859-1"?>\n', '<!DOCTYPE dblp SYSTEM "dblp.dtd">\n', '<dblp>\n']

paths = Path("dblp.xml")

def line_in_publication(line, publication):
    for i in publication:
        for j in range(len(line) - len(i) + 1):
            if (line[j:j + len(i)] == i):
                return [True, len(i)]
    return [False, 0]


with open(paths, "r") as f:
	fin = False
	num = 0
	numfichier = str(num)
	next_line = ''

	while fin == False:
		fichier = "fichier" + numfichier
		path = Path(fichier+".xml")
		with open(path, "a") as f1:
			counter = 0
			f1.write('<?xml version="1.0" encoding="ISO-8859-1"?>' + '\n')
			f1.write('<!DOCTYPE dblp SYSTEM "dblp.dtd">' + '\n')
			f1.write('<dblp>' + '\n')
			if next_line != '':
				f1.write(next_line)
			while counter < 1000000:
				line = f.readline()
				print(line)
				if line not in entete:
					word0 = line_in_publication(line, end)
					if line == '</dblp>':
						f1.write(line)
						fin = True	
						break
					if word0[0] == True:
						f1.write(line[:len(line)-word0[1]])
						f1.write('\n</dblp>')
						fin = True
						break
					word = line_in_publication(line, publications)
					word2 = line_in_publication(line, publications1)
					if word[0] == True:
						counter += 1
					if counter == 500000:
						if word2[0] == True:
							f1.write(line[0:word[1]])
							f1.write('\n</dblp>')
							next_line = line[word[1]:]
						else:
							f1.write(line)
							f1.write('</dblp>')
							break
					else:
						next_line = ''
						f1.write(line)

			num += 1
			numfichier = str(num)
	
	counter1 = 0

	with open(path,"r") as f2:
		while True:
			line = f2.readline()
			counter1 += 1
			if line == '</dblp>':
				break

	print(counter1, str(path))

	if counter1 == 4:
		try:
			os.remove(path)
		except OSError as e:
			print(e)
		else:
			print("File is deleted successfully")

# Exercice 1. Mise à disposition d’une API Web 

Réaliser en Python et en utilisant la bibliothèque bottle un serveur Web qui implémente l’API 
Web suivante :
- `/publications/{id}` : avec `id` l'identifiant d'une publication (à vous de choisir quels 
sont les identifiants), qui retourne la publication correspondante.
- `/publications` : qui retourne par défaut les 100 premières publications. La valeur 100 
peut être modifiée au moyen d’un paramètre d'url `limit`).
- `/authors/{name}` : avec `name` le nom d'un auteur, qui retourne des informations 
concernant un auteur : nombres de publications dont il est co-auteur, nombre de co-auteurs.
- `/authors/{name}/publications` : avec `name` le nom d'un auteur, qui liste les 
publications d'un auteur.
- `/authors/{name}/coauthors` : avec `name` le nom d'un auteur, qui liste les co-auteurs 
d'un auteur.
- `/search/authors/{searchString}` : avec `searchString` une chaine de caractères 
permettant de chercher un auteur. Cette route retourne la liste des auteurs dont le nom 
contient `searchString` (par exemple, /search/authors/w retourne la liste de tous les 
auteurs dont le nom contient un `w` ou un `W`). 
- `/search/publications/{searchString}`: avec `searchString` une chaine de 
caractères et qui retourne la liste des publications dont le titre contient `searchString`. La 
route accepte un paramètre d'url `filter` de la forme `key1:value1,key2:value2,...` 
afin d'affiner la recherche aux publications dont la clef `keyi` contient `valuei`. Ainsi, la 
recherche `/search/publications/robots?filter=author:Jean,journal:acm` 
doit retourner la liste des publications dont le titre contient `robots`, dont l'auteur contient 
`Jean` et publiées dans un journal contenant `acm`.
- `/authors/{name_origin}/distance/{name_destination}` : avec `name_origin` 
et `name_destination` deux noms d’auteurs, qui retourne la distance de collaboration 
entre les deux auteurs nommés. Cette distance est définie comme la longueur du plus petit 
chemin `(name_origin, auth1, auth2, ..., authX, name_destination)`, où 
`name_origine` et `auth1` sont co-auteurs, `auth1` et `auth2` sont co-auteurs, ... et 
`authX` et `name_destination` sont co-auteurs. En particulier deux co-auteurs sont à 
distance 1. En plus de retourner la distance, la réponse doit contenir un plus court chemin 
entre les deux auteurs.

L’API ainsi développée doit présenter les caractéristiques suivantes :
- Toutes les erreurs doivent avoir le même format.
- Chaque route doit être documentée, avec le format de retour, les erreurs possibles et une 
explications des paramètres.
- Chaque route qui retourne une liste, doit retourner au maximum 100 éléments et accepter 
des paramètre d'URL `start` et `count` qui permettent d'afficher `count` éléments, à partir 
du `start`-ième élément. Par exemple: `/search/authors/*` doit retourner les 100 
premiers auteurs, `/search/authors/*?start=100` affiche les 100 suivants, et `/
search/authors/*?start=200&count=2` affiche les 2 éléments suivants.
- Pour chaque route qui retourne une liste, les éléments retournés doivent pouvoir être triés 
par rapport à un champ donné dans un paramètre d'URL `order`. Par exemple: `/search/
publications/*?order=journal` affiche les 100 premières publications triées dans 
l'ordre alphabétique du nom du journal dans lequel elles apparaissent.

In [3]:
%%writefile run_api_web.py
'''
Created on November 22, 2022

@author: Thierry Ung, Jack Thay

Exercice 1 from MP2
'''
from bottle import *
from collections import OrderedDict
from json import *
from re import *
import xml.etree.cElementTree as ET # Element Tree written in C
from lxml import etree as ET

# XML file is really heavy, from the different computers we used for coding,
# only one was able to load dblp.xml and execute the code properly after 10 minutes
# file_name = "dblp.xml"

# fichier0.xml is a lighter xml file, so that we can ACTUALLY both code using our own computers.
file_name = "fichier0.xml"
p = ET.XMLParser(recover=True)
root = ET.parse(file_name, parser=p).getroot()


@route('/publications/<id>', method="GET")
def publications_id(id):
    """
    Fonction qui retourne toutes les informations de la publication par son titre.
    erreur possible:
    - titre de la publication pas trouvé
    """
    counter = 0
    info = ''

    for i in root.iter('title'):
        if i.text == id:
            for j in root[counter]:
                info += str(j.tag) + ": " + str(j.text) + "<br/>"
            info += '------------------------------------------------------------' + "<br/>"
            return info
        counter += 1

    abort(404, "Not found: '/publications/" + id + "'")


@route('/publications/', method="GET")
def publications():
    """
    Fonction qui retourne toutes les informations des limit (par defaut 100) premieres publications.
    paramètres possibles:
    - start: liste les publications apres start publications
    - count/limit: liste les count/limit publications suivantes
    - order: trie la liste des publications en fonction de order (si order=author trie par ordre alphabetique en fonction des auteurs)
    """
    limit = request.query.limit
    start = request.query.start
    count = request.query.count
    order = request.query.order

    if limit == "":
        limit = 100
    if start == "":
        start = 0
    if count != "" and int(count) <= 100:
        limit = count

    info = ""
    publications = []
    dico = {}

    if order != "":
        for i in range(int(start), int(start) + int(limit)):
            for j in root[i]:
                if j.tag == 'title':
                    titre = j.text
                    for k in root[i]:
                        if k.tag == order:
                            dico[titre] = k.text
    else:
        for i in range(int(start), int(start) + int(limit)):
            for j in root[i]:
                info += str(j.tag) + ": " + str(j.text) + "<br/>"
            info += '------------------------------------------------------------' + "<br/>"

    if len(dico) != 0:
        dico_sorted = OrderedDict(sorted(dico.items(), key=lambda t: t[1]))
        for keys in dico_sorted:
            publications.append(keys)
        for infos in publications:
            info += publications_id(infos)

    return info


@route('/authors/<name>', method="GET")
def authors_name(name):
    """
    Fonction qui retourne le nombre de publications et de coauthors d'un auteur.
    erreurs possibles:
    - nom de l'auteur pas trouvé
    """
    publications = []
    co_authors = []
    info = ''

    for i in root:
        for j in range(len(i)):
            if i[j].tag == "author":
                if i[j].text == name:
                    for k in range(len(i)):
                        if i[k].tag == "author":
                            if i[k].text != name and i[k].text not in co_authors:
                                co_authors.append(i[k].text)
                        if i[k].tag == "title":
                            if i[k].text not in publications:
                                publications.append(i[k].text)

    if len(publications) == 0 & len(co_authors) == 0:
        abort(404, "Not found: '/authors/" + name + "'")

    info += "Name of the author : " + str(name) + "<br/><br/>"
    info += "Number of publications : " + str(len(publications)) + "<br/><br/>"

    for i in range(len(publications)):
        info += str(publications[i]) + "<br/>"

    info += "<br/>" + "Number of co-authors : " + \
        str(len(co_authors)) + "<br/><br/>"

    for i in range(len(co_authors)):
        info += co_authors[i] + "<br/>"

    return info


@route('/authors/<name>/publications', method="GET")
def authors_name_publications(name):
    """
    Fonction qui retourne la liste des publications d'un auteur.
    erreurs possibles:
    - nom de l'auteur pas trouvé
    paramètres possibles:
    - start
    - count/limit
    - order
    """
    limit = request.query.limit
    start = request.query.start
    count = request.query.count
    order = request.query.order

    if limit == "":
        limit = 100
    if start == "":
        start = 0
    if count != "" and int(count) <= 100:
        limit = int(count)
    if order == "":
        order = 0

    info = ""
    counter = 0
    publications = []
    dico = {}

    if order == 0:
        for i in root:
            for j in range(len(i)):
                if i[j].tag == "author":
                    if i[j].text == name:
                        for k in range(len(i)):
                            if i[k].tag == "title":
                                if i[k].text not in publications:
                                    publications.append(i[k].text)
    else:
        for i in root:
            for j in range(len(i)):
                if i[j].tag == "author":
                    if i[j].text == name:
                        for k in range(len(i)):
                            if i[k].tag == order:
                                for l in range(len(i)):
                                    if i[l].tag == "title":
                                        if i[k].text not in dico:
                                            dico[i[k].text] = i[l].text

    if len(dico) != 0:
        dico_sorted = OrderedDict(sorted(dico.items(), key=lambda t: t[0]))
        for keys in dico_sorted:
            publications.append(dico_sorted[keys])

    if len(publications) == 0 or len(publications) <= int(start):
        abort(404, "Not found: '/authors/" + name + "/publications'")

    info = "Publications of author " + str(name) + " :<br/><br/>"

    for i in range(int(start), len(publications)):
        if counter < int(limit):
            info += publications[i] + "<br/>"
            counter += 1

    return info


@route('/authors/<name>/coauthors', method="GET")
def authors_name_coauthors(name):
    """
    Fonction qui retourne la liste des coauteurs d'un auteur.
    erreurs possibles:
    - nom de l'auteur pas trouvé
    paramètres possibles:
    - start
    - count/limit
    - order
    """
    limit = request.query.limit
    start = request.query.start
    count = request.query.count
    order = request.query.order

    if limit == "":
        limit = 100
    if start == "":
        start = 0
    if count != "" and int(count) <= 100:
        limit = int(count)
    if order == "":
        order = 0

    info = ""
    counter = 0
    co_authors = []
    dico = {}

    if order == 0:
        for i in root:
            for j in range(len(i)):
                if i[j].tag == "author":
                    if i[j].text == name:
                        for k in range(len(i)):
                            if i[k].tag == "author":
                                if i[k].text != name and i[k].text not in co_authors:
                                    co_authors.append(i[k].text)
    else:
        for i in root:
            for j in range(len(i)):
                if i[j].tag == "author":
                    if i[j].text == name:
                        for k in range(len(i)):
                            if i[k].tag == "author":
                                if i[k].text != name and i[k].text not in dico:
                                    for l in range(len(i)):
                                        if i[l].tag == order:
                                            if order == "author":
                                                if i[l].text == i[k].text:
                                                    dico[i[k].text] = i[k].text
                                            else:
                                                dico[i[k].text] = i[l].text

    if len(dico) != 0:
        dico_sorted = OrderedDict(sorted(dico.items(), key=lambda t: t[1]))
        for keys in dico_sorted:
            co_authors.append(keys)

    if len(co_authors) == 0:
        return "No co-authors"

    if int(start) >= len(co_authors):
        abort(404, "Not found: '/authors/" + name + "/coauthors'")

    info = "Co-authors of author " + str(name) + " :<br/><br/>"

    for i in range(int(start), len(co_authors)):
        if counter < limit:
            info += co_authors[i] + "<br/>"
            counter += 1

    return info


def mot_in_string(string, mot):
    """
    Fonction qui prend en paramètre une chaîne de caractères et un mot.
    Elle renvoie un booléen à True si la chaîne de caractères contient le mot, False sinon.
    """
    string = string.lower()
    mot = mot.lower()

    for i in range(len(string) - len(mot) + 1):
        if string[i: i + len(mot)] == mot:
            return True

    return False


@route('/search/authors/<searchString>', method="GET")
def search_authors_searchString(searchString):
    """
    Fonction qui retourne la liste des auteurs contenant la chaine searchString dans leur nom.
    erreurs possibles:
    - aucun auteur trouvé comportant searchString
    paramètres possibles:
    - start
    - count/limit
    - order
    """
    limit = request.query.limit
    start = request.query.start
    count = request.query.count
    order = request.query.order

    if limit == "":
        limit = 100
    if start == "":
        start = 0
    if count != "" and int(count) <= 100:
        limit = int(count)
    if order == "":
        order = 0

    info = ""
    counter = 0
    authors = []
    dico = {}

    if order == 0:
        for i in root:
            for j in range(len(i)):
                if i[j].tag == "author":
                    if mot_in_string(str(i[j].text), str(searchString)) and i[j].text not in authors:
                        authors.append(i[j].text)
    else:
        for i in root:
            for j in range(len(i)):
                if i[j].tag == "author":
                    if mot_in_string(str(i[j].text), str(searchString)) and i[j].text not in dico:
                        for k in range(len(i)):
                            if i[k].tag == order:
                                if order == "author":
                                    if i[k].text == i[j].text:
                                        dico[i[j].text] = i[j].text
                                else:
                                    dico[i[j].text] = i[k].text

    if len(dico) != 0:
        dico_sorted = OrderedDict(sorted(dico.items(), key=lambda t: t[1]))
        for keys in dico_sorted:
            authors.append(keys)

    if len(authors) == 0:
        return "No author found"

    if len(authors) <= int(start):
        abort(404, "Not found: '/authors/"+searchString+"'")

    info = "Authors with the sub-sequence : " + \
        str(searchString) + "<br/><br/>"

    for i in range(int(start), len(authors)):
        if counter < int(limit):
            info += "<author>"
            info += authors[i] + "<br/>"
            info += "</author>"
            counter += 1

    return info


@route('/search/publications/<searchString>', method="GET")
def search_publications_searchString(searchString):
    """
    Fonction qui retourne la liste des publications contenant la chaine searchString dans leur titre.
    erreurs possibles:
    - aucune publication trouvé comportant searchString
    paramètres possibles:
    - start
    - count/limit
    - order
    - filter : permet de filtrer par balise (search/publications/covid?filter=author:J,journal:acm 
               va retourner la liste des publications dont le titre contient `covid`, dont l'auteur contient 
              `J` et publiées dans un journal contenant `acm`).
    """
    limit = 100
    start = request.query.start
    count = request.query.count
    order = request.query.order

    if start == "":
        start = 0
    if count != "" and int(count) <= 100:
        limit = int(count)
    if order == "":
        order = 0

    info = ""
    counter = 0
    filtre = "{" + request.query.filter + "}"
    tmp = ""
    quote = False

    for i in filtre:
        if i.isalnum():
            if not quote:
                tmp += '"'
                quote = True
        else:
            if quote:
                tmp += '"'
                quote = False
        tmp += i

    dictionaire = loads(tmp)
    publications = []
    dico = {}

    if len(dictionaire) == 0:
        if order == 0:
            for i in root:
                for j in range(len(i)):
                    if i[j].tag == "title":
                        if mot_in_string(str(i[j].text), str(searchString)) and i[j].text not in publications:
                            publications.append(i[j].text)
                            print(i[j].text)
        else:
            for i in root:
                for j in range(len(i)):
                    if i[j].tag == "title":
                        if mot_in_string(str(i[j].text), str(searchString)) and i[j].text not in dico:
                            for k in range(len(i)):
                                if i[k].tag == order:
                                    if order == "title":
                                        if i[k].text == i[j].text:
                                            dico[i[j].text] = i[j].text
                                    else:
                                        dico[i[j].text] = i[k].text
    else:
        if order == 0:
            for i in root:
                for j in range(len(i)):
                    if i[j].tag == "title":
                        if mot_in_string(str(i[j].text), str(searchString)) == True:
                            counter = 0
                            for key in dictionaire.keys():
                                for k in range(len(i)):
                                    if i[k].tag == key:
                                        if mot_in_string(str(i[k].text), dictionaire[key]) == True:
                                            counter += 1
                            if counter == len(dictionaire) and i[j].text not in publications:
                                publications.append(i[j].text)
        else:
            for i in root:
                for j in range(len(i)):
                    if i[j].tag == "title":
                        if mot_in_string(str(i[i].text), str(searchString)) == True:
                            counter = 0
                            for key in dictionaire.keys():
                                for k in range(len(i)):
                                    if i[k].tag == key:
                                        if mot_in_string(str(i[k].text), dictionaire[key]) == True:
                                            counter += 1
                            if counter == len(dictionaire) and i[j].text not in dico:
                                for l in range(len(i)):
                                    if i[l].tag == order:
                                        if order == "title":
                                            if i[l].text == i[j].text:
                                                dico[i[j].text] = i[j].text
                                        else:
                                            dico[i[j].text] = i[l].text

    if len(dico) != 0:
        dico_sorted = OrderedDict(sorted(dico.items(), key=lambda t: t[1]))
        for keys in dico_sorted:
            publications.append(keys)

    if len(publications) == 0:
        return "No publications found"

    if int(start) >= len(publications):
        abort(404, "Not found: '/publications/"+searchString+"'")

    info = "Publications with the sub-sequence : " + \
        str(searchString) + "<br/><br/>"

    for i in range(int(start), len(publications)):
        if counter < int(limit):
            counter += 1
            info += "<publication>"
            info += publications[i] + "<br/>"
            info += "</publication>"

    return info


def min_valeur(dico):
    """
    Fonction qui prend en paramètre un dictionnaire et retourne la plus petite valeur 
    du dictionnaire et le chemin associé à cette valeur
    """
    val = float("inf")
    key = ''

    for cle, valeur in dico.items():
        if valeur[0] < val:
            val = valeur[0]
            key = cle

    if val == float("inf"):
        return (val, '')
    else:
        return (val, dico[key][1])


def distance(name_origin, name_destination, dictionnaire, chemin, coauthors, counter):
    """
    Fonction qui va être appelée récursivement sur les co-auteurs de l’auteur d’origine. La distance augmente dès lors que l’on appelle la fonction sur un co-auteur. 
    Si un auteur n’a plus de co-auteur, c’est que l’on ne peut pas trouver l’auteur de destination à partir de ce chemin, on renvoie donc une distance infinie. 
    Si on le trouve on renvoie la plus petite distance des chemins de chacun de ses co-auteurs ainsi que le chemin correspondant. 
    Pour trouver la plus petite distance d’un ensemble de chemin, on utilise la fonction min_valeur. 
    Le plus petit chemin trouvé est stocké et chaque parcours de chemin qui est plus grand que le plus petit chemin actuel renvoie 
    immédiatement un chemin de distance infini pour optimiser le temps d’exécution de la fonction. 
    De plus, un auteur ne compte pas un autre auteur comme son co-auteur si cet auteur a déjà parcouru ses co-auteurs.
    """
    chemin_copy = chemin.copy()
    chemin_copy.append(name_origin)
    coauthors_copy = coauthors.copy()
    co_authors = []

    for i in root:
        for j in range(len(i)):
            if i[j].tag == "author":
                if i[j].text == name_origin:
                    for k in range(len(i)):
                        if i[k].tag == "author":
                            if i[k].text != name_origin and i[k].text not in co_authors and i[k].text not in chemin and i[k].text not in coauthors_copy and i[k].text not in dictionnaire.keys():
                                co_authors.append(i[k].text)

    coauthors_copy = co_authors

    if len(co_authors) == 0:
        return [float("inf"), ""]
    else:
        if name_destination in co_authors and len(chemin) == 0:
            return [1, chemin_copy]
        else:
            if name_destination in co_authors:
                if len(chemin_copy) < counter:
                    counter = len(chemin_copy)
                return [1, chemin_copy]
            else:
                if len(chemin_copy) < counter:
                    if len(co_authors) > 1 and name_origin not in dictionnaire.keys():
                        dictionnaire[name_origin] = chemin_copy
                        dico = {}
                        for authors in co_authors:
                            dico[authors] = [1, ""]
                        for authors in co_authors:
                            res = distance(authors, name_destination, dictionnaire,
                                           dictionnaire[name_origin], coauthors_copy, counter)
                            dico[authors][0] += res[0]
                            dico[authors][1] = res[1]
                        return min_valeur(dico)
                    else:
                        dico = {}
                        for authors in co_authors:
                            dico[authors] = [1, ""]
                        for authors in co_authors:
                            res = distance(
                                authors, name_destination, dictionnaire, chemin_copy, coauthors_copy, counter)
                            dico[authors][0] += res[0]
                            dico[authors][1] = res[1]
                        return min_valeur(dico)
                else:
                    return [float("inf"), ""]


@route('/authors/<name_origin>/distance/<name_destination>', method="GET")
def authors_name_origin_distance_name_destination(name_origin, name_destination):
    """
    Fonction qui retourne la distance minimale entre deux auteurs et liste le chemin d'auteur.
    erreurs possibles:
    - aucun chemin trouvé entre les deux auteurs fournits
    """
    min_distance, min_chemin = distance(
        name_origin, name_destination, {}, [], [], float("inf"))

    if min_distance == float("inf"):
        return "No path from author " + str(name_origin) + " to author " + str(name_destination) + "."
    else:
        return "The length of the smallest path " + str(min_chemin) + " between author " + str(name_origin) + " and author " + str(name_destination) + " is " + str(min_distance) + "."


run(host='localhost', port=8080, debug=True)

Overwriting run_api_web.py


In [2]:
!wt python run_api_web.py

# Exercice 2. Test unitaire d’une API Web 

Réaliser en Python et en utilisant au choix la bibliothèque unittest ou la bibliothèque pytest un programme qui teste le bon 
fonctionnement de l'API Web développée à l'exercice 1.

In [14]:
%%writefile test_file.py
'''
Created on November 22, 2022

@author: Thierry Ung, Jack Thay

Exercice 2 from MP2
'''
from requests import *
from json import *
import pytest


server_ip = "localhost"
server_port = 8080

## Tests for publications ##

def test_publications_not_404():
    '''
    Test if the API is actually running
    '''
    r = get(f"http://{server_ip}:{server_port}/publications/")
    # assert r.text != None
    # r.text != Note" is too easy for a test, r.text will never be empty, even if error 404
    condition = "author" # because the 6 first character from r.text should theorically be "author"
    condition2 = "title"
    assert r.text[:6] == condition or condition2

def test_publications_limit():
    '''
    Test if the "?limit=X" feature work
    '''
    r = get(f"http://{server_ip}:{server_port}/publications/?limit=5")
    condition = "author"
    assert r.text[:6] == condition

def test_publications_start():
    '''
    Test if the "?start=X" feature work
    '''
    r = get(f"http://{server_ip}:{server_port}/publications/?start=2")
    condition = "author"
    condition2 = "title:" # There might be a possibility the first 6 letters are "title:"
    assert r.text[:6] == condition or condition2

def test_publications_order():
    '''
    Test if the "?order=X" feature work
    '''
    r = get(f"http://{server_ip}:{server_port}/publications/?order=journal")
    condition = "author"
    condition2 = "title:" # There might be a possibility the first 6 letters are "title:"
    assert r.text[:6] == condition or condition2
    
def test_publications_count():
    '''
    Test if the "?count=X" feature work
    '''
    r = get(f"http://{server_ip}:{server_port}/publications/?count=5")
    condition = "author"
    condition2 = "title:" # There might be a possibility the first 6 letters are "title:"
    assert r.text[:6] == condition or condition2

def test_publications_id1():
    '''
    1st test for a given title
    Title used : Spectre Attacks: Exploiting Speculative Execution.
    '''
    r = get(f"http://{server_ip}:{server_port}/publications/Spectre Attacks: Exploiting Speculative Execution.")
    condition = "author: Paul Kocher"
    assert r.text[:19] == condition
    #assert loads(r.text) == ("author: Paul Kocher<br/>author: Daniel Genkin<br/>author: Daniel Gruss<br/>author: Werner Haas 0004<br/>author: Mike Hamburg<br/>author: Moritz Lipp<br/>author: Stefan Mangard<br/>author: Thomas Prescher 0002<br/>author: Michael Schwarz 0001<br/>author: Yuval Yarom<br/>title: Spectre Attacks: Exploiting Speculative Execution.<br/>journal: meltdownattack.com<br/>year: 2018<br/>ee: https://spectreattack.com/spectre.pdf<br/>")

def test_publications_id2():
    '''
    2nd test for a given title
    Title used : Meltdown
    '''
    r = get(f"http://{server_ip}:{server_port}/publications/Meltdown")
    condition = "author: Moritz Lipp"
    print(r.text)
    assert r.text[:19] == condition

def test_publications_id3():
    '''
    3rd test for a given title
    Title used : An Evaluation of Object-Oriented DBMS Developments: 1994 Edition.
    '''
    r = get(f"http://{server_ip}:{server_port}/publications/An Evaluation of Object-Oriented DBMS Developments: 1994 Edition.")
    condition = "author: Frank Manola"
    assert r.text[:20] == condition
    
## Tests for authors ##

def test_authors_not_404():
    '''
    Test if the API is actually running with the "authors" route
    Author used : Daniel Genkin
    '''
    r = get(f"http://{server_ip}:{server_port}/authors/Daniel Genkin")
    condition = "Name of the author" # If succesful, first words on the webpage should be this
    assert r.text[:18] == condition
    
def test_authors_publications():
    '''
    Test if the API is actually running with the "authors"+"publications" route
    Author used : Daniel Genkin
    '''
    r = get(f"http://{server_ip}:{server_port}/authors/Daniel Genkin/publications")
    condition = "Publications of author Daniel Genkin" # If succesful, first words on the webpage should be this
    assert r.text[:36] == condition
    
def test_authors_coauthors():
    '''
    Test if the API is actually running with the "authors"+"coauthors" route
    Author used : Daniel Genkin
    '''
    r = get(f"http://{server_ip}:{server_port}/authors/Daniel Genkin/coauthors")
    condition = "Co-authors of author Daniel Genkin" # If succesful, first words on the webpage should be this
    assert r.text[:34] == condition
    
## Tests for search ##

def test_search_authors():
    '''
    Test if the API is actually running with the "search"+"authors" route
    String used : Daniel
    '''
    r = get(f"http://{server_ip}:{server_port}/search/authors/Daniel")
    condition = "Authors with the sub-sequence" # If succesful, first words on the webpage should be this
    assert r.text[:29] == condition
    
def test_search_publications():
    '''
    Test if the API is actually running with the "search"+"publications" route
    String used : COVID
    '''
    r = get(f"http://{server_ip}:{server_port}/search/publications/COVID")
    condition = "Publications with the sub-sequence" # If succesful, first words on the webpage should be this
    assert r.text[:34] == condition
    
def test_search_no_result():
    '''
    Test for when the "search" route return no results
    String used : noone
    '''
    r = get(f"http://{server_ip}:{server_port}/search/publications/noone")
    condition = "No publications found" # If failed, first words on the webpage should be this
    assert r.text[:21] == condition
    
## Tests for distance ##

def test_distance_not_404():
    '''
    Test if the API is actually running with the "distance" route
    Authors used : Paul Kocher and Daniel Genkin
    '''
    r = get(f"http://{server_ip}:{server_port}/authors/Paul Kocher/distance/Daniel Genkin")
    condition = "The length of the smallest path" # If succesful, first words on the webpage should be this
    assert r.text[:31] == condition
    
def test_distance_no_result():
    '''
    Test if the API is actually running with the "distance" route
    Authors used : Paul Kocher and Daniel Genkin
    '''
    r = get(f"http://{server_ip}:{server_port}/authors/Paul Kocher/distance/Frank Manola")
    condition = "No path from author" # If failed, first words on the webpage should be this
    assert r.text[:19] == condition

Overwriting test_file.py


In [ ]:
!python -m pytest test_file.py -vv

# Exercice 3. Site Web d’utilisation d’une API Web 

Réaliser en Python et en utilisant la bibliothèque bottle un serveur Web qui utilise l’API Web 
développée à l’exercice 1 pour proposer à l’utilisateur une interface Web graphique qui lui 
permette d’obtenir en entrant les informations pertinentes dans un formulaire Web : 
- la liste complète des publications et la liste complète des coauteurs d’un auteur, possiblement 
triées alphabétiquement. Cet auteur peut au préalable être recherché via une sous-séquence 
de caractères apparaissant dans son nom. 
- La distance entre deux auteurs. Ces auteurs peuvent au préalable être recherchés chacun 
via une sous-séquence de caractères apparaissant dans leur nom.

In [ ]:
%%writefile run_interface_web.py
'''
Created on November 22, 2022

@author: Thierry Ung, Jack Thay

Exercice 3 from MP2
'''

from bottle import *
from json import *
from requests import get

server_ip = "localhost"
server_port = 8080


@route("/author")
def author():
    """
    Fonction qui affiche une interface graphique pour entrer un auteur via une sous-séquence 
    de caractères apparaissant dans son nom ou recherche directe de l'auteur.
    """
    return '''
         <form action="/author" method="post">
         Search for an author that contains the sub-sequence : <input name="sequence" type="text" /></br></br>
         Apply an order : <input name="order" type="text" />
         <input value="Submit" type="submit" />
         </form>
         or</br></br>
         <form action="/doauthor" method="post">
         Search for the author : <input name="author" type="text" />
         <input value="Submit" type="submit" />
         </form>
         '''


@route("/author", method='POST')
def author():
    """
    Fonction qui affiche une interface graphique pour entrer un author.
    """
    s = request.forms['sequence']
    order = request.forms['order']

    r = get(
        f"http://{server_ip}:{server_port}/search/authors/{s}?order={order}")

    liste = r.text

    return '''
         <form action="/doauthor" method="post">
         Search for the author : <input name="author" type="text" />
         <input value="Submit" type="submit" />
         </form>
         </br>
         </br>
         ''', liste


@route("/doauthor", method='POST')
def do_author():
    """
    Fonction qui appelle l'api avec l'author rentré pour obtenir la liste des coauthors et des publications de l'author.
    """
    s = request.forms['author']

    r = get(f"http://{server_ip}:{server_port}/authors/{s}/coauthors")
    r1 = get(f"http://{server_ip}:{server_port}/authors/{s}/publications")

    liste = r.text
    liste1 = r1.text

    return liste, '''</br></br>''', liste1


@route("/distance")
def distance():
    """
    Fonction qui affiche une interface graphique pour entrer deux auteur via une sous-séquence 
    de caractères apparaissant dans leur nom ou recherche directe des auteurs.
    """
    return '''
         <form action="/distance" method="post">
         Search for the first author that contains the sub-sequence : <input name="author1" type="text" /></br></br>
         Search for the second author that contains the sub-sequence : <input name="author2" type="text" />
         <input value="Submit" type="submit" />
         </form>
         or</br></br>
         <form action="/doinputdistance" method="post">
         First author: <input name="author3" type="text" /></br></br>
         Second author: <input name="author4" type="text" />
         <input value="Submit" type="submit" />
         </form>
         '''


@route("/distance", method='POST')
def distance():
    """
    Fonction qui affiche une interface graphique pour entrer deux auteur.
    """
    author1 = request.forms['author1']
    author2 = request.forms['author2']

    r = get(f"http://{server_ip}:{server_port}/search/authors/{author1}")
    r1 = get(f"http://{server_ip}:{server_port}/search/authors/{author2}")

    liste = r.text
    liste1 = r1.text

    return '''
         <form action="/doinputdistance" method="post">
         First author: <input name="author3" type="text" /></br></br>
         Second author: <input name="author4" type="text" />
         <input value="Submit" type="submit" />
         </form>
         </br>
         </br>
         ''', liste, '''</br></br>''', liste1


@route("/doinputdistance", method='POST')
def do_input_distance():
    """
    Fonction qui appelle l'api avec les deux auteurs entré pour obtenir la distance minimale entre ces deux auteurs ainsi que le chemin.
    """
    author1 = request.forms['author3']
    author2 = request.forms['author4']

    r = get(
        f"http://{server_ip}:{server_port}/authors/{author1}/distance/{author2}")

    liste = r.text

    return liste


run(host='localhost', port=8081)

In [ ]:
!wt python run_interface_web.py